# ResNet with pytorch

- load data
- build model
- train and test
- transfer learning

In [3]:
import torch
import torch.nn as nn
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)
        
def plot_image(img, label, name):
    fig = plt.figure()
    for i in range(6):
        plt.subplot(2,3,i+1)
        plt.tight_layout()
        plt.imshow(img[1][0]*0.3081 + 0.1307, cmap='gray', interpolation='none')
        plt.title("{}:{}".format(name, label[i].item()))
        plt.xticks([])
        plt.yticks([])
    plt.show

In [4]:
import  os, glob
import  random, csv
from    torch.utils.data import Dataset, DataLoader
from    torchvision import transforms
from    PIL import Image

class Pokemon(Dataset):
    def __init__(self, root, resize, mode):
        super(Pokemon, self).__init__()
        
        self.root = root
        self.resize = resize
        
        self.name2label = {}
        for name in sorted(os.listdir(os.path.join(root))):
            if not os.path.isdir(os.path.join(root, name)):
                continue
            
            self.name2label[name] = len(self.name2label.keys())
        
        self.images, self.labels = self.load_csv('images.csv')
        
        if mode == 'train':
            self.images = self.images[:int(0.6*len(self.images))]
            self.labels = self.labels[:int(0.6*len(self.labels))]
        elif mode == 'val':
            self.images = self.images[int(0.6*len(self.images)):int(0.8*len(self.images))]
            self.labels = self.labels[int(0.6*len(self.labels)):int(0.8*len(self.labels))]
        elif mode == 'test':
            self.images = self.images[int(0.8*len(self.images)):]
            self.labels = self.labels[int(0.8*len(self.labels)):]
    
    def load_csv(self, filename):
        if not os.path.exists(os.path.join(self.root, filename)):
            images = []
            for name in self.name2label.keys():
                images += glob.glob(os.path.join(self.root, name, '*.png'))
                images += glob.glob(os.path.join(self.root, name, '*.jpg'))
                images += glob.glob(os.path.join(self.root, name, '*.jpeg'))
            random.shuffle(images)
            with open(os.path.join(self.root, filename), mode='w', newline='') as f:
                writer = csv.writer(f)
                for img in images:
                    name = img.split(os.sep)[-2]
                    label = self.name2label[name]
                    writer.writerow([img,label])
                print('writen into csv file:', filename)
                
        images, labels = [], []
        with open(os.path.join(self.root, filename)) as f:
            reader = csv.reader(f)
            for row in reader:
                img,label = row
                label = int(label)
                images.append(img)
                labels.append(label)
        assert len(images) == len(labels)
        return images, labels
    def __len__(self):
        return len(self.images)
    
    def denormalize(self, x_hat):
        
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        # x_hat = (x-mean)/std
        # x = x_hat*std = mean
        # x: [c, h, w]
        # mean: [3] => [3, 1, 1]
        mean = torch.tensor(mean).unsqueeze(1).unsqueeze(1)
        std = torch.tensor(std).unsqueeze(1).unsqueeze(1)
        # print(mean.shape, std.shape)
        x = x_hat * std + mean
        return x
    
    def __getitem__(self, idx):
        img, label = self.images[idx], self.labels[idx]
        tf = transforms.Compose([
            lambda x:Image.open(x).convert('RGB'), # string path= > image data
            transforms.Resize((int(self.resize*1.25), int(self.resize*1.25))),
            transforms.RandomRotation(15),
            transforms.CenterCrop(self.resize),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        
        img = tf(img)
        label = torch.tensor(label)
        return img,label
                

In [15]:
# build model
from    torch.nn import functional as F
class ResBlk(nn.Module):
    def __init__(self, inChannel, outChannel, stride=1):
        super(ResBlk, self).__init__()
        self.conv1 = nn.Conv2d(inChannel, outChannel, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(outChannel)
        self.conv2 = nn.Conv2d(outChannel, outChannel, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(outChannel)
        
        self.extra = nn.Sequential()
        if outChannel!=inChannel:
            self.extra = nn.Sequential(
            nn.Conv2d(inChannel, outChannel, kernel_size=1, stride=stride),
            nn.BatchNorm2d(outChannel))
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.extra(x) + out
        out = F.relu(out)
        return out

In [21]:
class ResNet18(nn.Module):
    def __init__(self, num_class):
        super(ResNet18, self).__init__()
        self.conv1 = nn.Sequential(
        nn.Conv2d(3,16,kernel_size=3,stride=3,padding=0),
        nn.BatchNorm2d(16))
        
        self.blk1 = ResBlk(16,32,stride=3)
        self.blk2 = ResBlk(32,64,stride=3)
        self.blk3 = ResBlk(64,128,stride=2)
        self.blk4 = ResBlk(128,256,stride=2)
        
        self.outlayer = nn.Linear(256*3*3, num_class)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.blk1(x)
        x = self.blk2(x)
        x = self.blk3(x)
        x = self.blk4(x)
        x = x.view(x.size(0), -1)
        x = self.outlayer(x)
        return x

In [35]:
import torch
from torch import optim, nn
import visdom
batchsz = 32
lr = 1e-3
epochs = 10

trainData = Pokemon('./data/pokemon', 224, mode='train')
valData = Pokemon('./data/pokemon', 224, mode='val')
testData = Pokemon('./data/pokemon', 224, mode='test')

trainLoader = DataLoader(trainData, batch_size=batchsz, shuffle=True,
                        num_workers=4)
valLoader = DataLoader(valData, batch_size=batchsz, shuffle=False,
                      num_workers=2)
testLoader = DataLoader(testData, batch_size=batchsz, shuffle=False,
                       num_workers=2)

model = ResNet18(5)
optimizer = optim.Adam(model.parameters(), lr=lr)
criteon = nn.CrossEntropyLoss()
viz = visdom.Visdom()
viz.line([0], [-1], win='loss', opts=dict(title='loss'))
viz.line([0], [-1], win='val_acc', opts=dict(title='val_acc'))
def evaluate(model, loader):
    correct = 0
    total = len(loader.dataset)
    for x,y in loader:
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred,y).sum().float().item()
    return correct/total

best_acc, best_epoch = 0,0
global_step = 0
for epoch in range(epochs):

        for step, (x,y) in enumerate(trainLoader):
            model.train()
            logits = model(x)
            loss = criteon(logits, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            viz.line([loss.item()], [global_step], win='loss', update='append')
            print(loss.item(), global_step)
            global_step += 1

        if epoch % 1 == 0:

            val_acc = evalute(model, valLoader)
            if val_acc> best_acc:
                best_epoch = epoch
                best_acc = val_acc

                torch.save(model.state_dict(), 'best.mdl')

                viz.line([val_acc], [global_step], win='val_acc', update='append')
print('best_acc:', best_acc,'best_epoch:', best_epoch)

model.load_state_dict(torch.load('best.mdl'))
test_acc = evalute(model, testLoader)
print('test_acc:',test_acc)
        


1.5182881355285645 0
2.103452205657959 1
2.068197727203369 2
1.9896528720855713 3
2.0654709339141846 4
0.7708738446235657 5
2.038616180419922 6
0.6397014856338501 7
0.9876345992088318 8
0.882544994354248 9
1.1049165725708008 10
0.6454030275344849 11
0.9028308391571045 12
0.7742918133735657 13
0.46328219771385193 14
0.5010548233985901 15
0.9106078147888184 16


/Users/striverwang/anaconda3/envs/dl/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


0.26927223801612854 17
0.2981182336807251 18
0.9401915669441223 19
0.8364970088005066 20
0.8666273951530457 21


/Users/striverwang/anaconda3/envs/dl/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +
/Users/striverwang/anaconda3/envs/dl/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


0.20236420631408691 22
0.5642319321632385 23
0.6349573135375977 24
0.40248703956604004 25
0.4329265058040619 26
0.6033247113227844 27
0.8497241735458374 28
0.3229106068611145 29
0.26927077770233154 30
0.19804391264915466 31
0.15070338547229767 32
0.6603262424468994 33
0.3132723569869995 34
0.3251098394393921 35
0.3701622784137726 36
0.5432785153388977 37
1.3515344858169556 38
0.9648956060409546 39
0.6611078381538391 40
0.5435547232627869 41
0.662379801273346 42
0.2745145559310913 43


...

0.11700162291526794 198
0.1360456794500351 199
0.07049642503261566 200
0.06530829519033432 201
0.13503466546535492 202
0.13573579490184784 203
0.3640984296798706 204
0.20868128538131714 205
0.06068388372659683 206
0.4632214307785034 207
0.10328789055347443 208
0.4642248749732971 209


/Users/striverwang/anaconda3/envs/dl/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


0.33785462379455566 210
0.11723791062831879 211
0.02525440603494644 212
0.11242695152759552 213
0.04972611367702484 214
0.22670121490955353 215
1.3936501741409302 216
0.1763879805803299 217
0.1639140397310257 218
0.06320852041244507 219


/Users/striverwang/anaconda3/envs/dl/lib/python3.7/site-packages/PIL/Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


best_acc: 0.8755364806866953 best_epoch: 8
test_acc: 0.8497854077253219


In [42]:
resize = 224
tf = transforms.Compose([
            lambda x:Image.open(x).convert('RGB'), # string path= > image data
            transforms.Resize((int(resize*1.25), int(resize*1.25))),
            transforms.RandomRotation(15),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
img = tf('./data/pokemon/bulbasaur/00000001.png')

In [64]:
from torch.autograd import Variable
pred = model(img.view(1,3,224,224)).argmax(dim=1)
int(pred.numpy())

0

In [65]:
trainData.name2label

{'bulbasaur': 0, 'charmander': 1, 'mewtwo': 2, 'pikachu': 3, 'squirtle': 4}